<a href="https://colab.research.google.com/github/akshats911/Fake-News-Classifier/blob/main/fakeNEWSclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
!import -q kaggle

/bin/bash: import: command not found


In [3]:
!pip install -q kaggle

In [4]:
cd /root

/root


In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [6]:
!chmod 600 /root/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [7]:
!kaggle competitions download -c 'Fake News'

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [9]:
import sys,csv
csv.field_size_limit(sys.maxsize)
df = pd.read_csv('/root/train.csv', engine="python",error_bad_lines=False)

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df = df.dropna()

In [11]:
X = df.drop('label',axis=1)
X.shape

(18285, 4)

In [12]:
Y = df['label']
Y.shape

(18285,)

In [13]:
voc_size = 5000

In [14]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [15]:
messages = X.copy()
messages.reset_index(inplace=True)

In [16]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
import re 
from nltk.corpus import stopwords

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [21]:
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]'," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ lm.lemmatize(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [22]:
onehot_repr = [ one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2940, 3305, 3995, 4675, 690, 2533, 1313, 1805, 4528, 841],
 [3553, 2257, 4736, 3672, 1937, 2289, 420],
 [1276, 1386, 2637, 4545],
 [1193, 3385, 3620, 967, 1390, 2362],
 [1967, 1937, 3210, 3015, 2491, 2331, 1937, 854, 159, 3244],
 [1305,
  2623,
  3634,
  920,
  2381,
  3700,
  1208,
  173,
  3318,
  3918,
  3075,
  3942,
  3355,
  1130,
  420],
 [1753, 120, 3323, 2847, 395, 4129, 2848, 3966, 4589, 4481, 3186],
 [294, 2135, 416, 2356, 4832, 3543, 3700, 4116, 4589, 4481, 3186],
 [2527, 2409, 903, 212, 794, 38, 1112, 3543, 3700, 3887],
 [2865, 2316, 4728, 1793, 3355, 2208, 3916, 2245],
 [341, 4318, 524, 1566, 3751, 1249, 119, 1209, 697, 3124, 2670],
 [967, 1884, 690, 4919, 3700, 4832],
 [819, 4288, 4049, 4496, 3591, 4026, 1804, 2805, 1625],
 [2606, 1371, 4862, 605, 3147, 2757, 1763, 4589, 4481, 3186],
 [4672, 512, 2489, 2792, 4617, 4589, 4481, 3186],
 [643, 2928, 4342, 151, 3151, 2714, 4273, 2636, 3319, 455],
 [4502, 3294, 2257],
 [947, 3000, 3078, 1583, 3700, 390, 3659, 420],
 [667, 22

In [23]:
maxlen=1
for i in onehot_repr:
  if len(i)>maxlen:
    maxlen=len(i)
maxlen

47

In [24]:
sent_len = 50
embedded_docs = pad_sequences(onehot_repr, padding="pre", maxlen=sent_len)
embedded_docs          

array([[   0,    0,    0, ..., 1805, 4528,  841],
       [   0,    0,    0, ..., 1937, 2289,  420],
       [   0,    0,    0, ..., 1386, 2637, 4545],
       ...,
       [   0,    0,    0, ..., 4589, 4481, 3186],
       [   0,    0,    0, ..., 2093, 3686, 4354],
       [   0,    0,    0, ..., 3095, 2729, 3419]], dtype=int32)

In [25]:
embd_features = 80
model = Sequential([
    Embedding(
        voc_size, embd_features,input_length = sent_len
    ),
    LSTM(100),
    Dense(1,activation="sigmoid")]
)

model.compile(
    loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 80)            400000    
                                                                 
 lstm (LSTM)                 (None, 100)               72400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 472,501
Trainable params: 472,501
Non-trainable params: 0
_________________________________________________________________


In [26]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [27]:
X_final.shape, Y_final.shape

((18285, 50), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [29]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 11ms/step - loss: 0.2965 - accuracy: 0.8682 - val_loss: 0.1913 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1289 - accuracy: 0.9493 - val_loss: 0.2026 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0849 - accuracy: 0.9700 - val_loss: 0.2331 - val_accuracy: 0.9188
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0522 - accuracy: 0.9829 - val_loss: 0.2762 - val_accuracy: 0.9120
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0350 - accuracy: 0.9894 - val_loss: 0.3351 - val_accuracy: 0.9110
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0155 - accuracy: 0.9955 - val_loss: 0.4238 - val_accuracy: 0.9114
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0084 - accuracy: 0.9979 - val_loss: 0.4422 - val_accuracy: 0.9094
Epoch

In [30]:
Y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [31]:
X_test

array([[   0,    0,    0, ..., 3088, 4863, 1100],
       [   0,    0,    0, ..., 4589, 4481, 3186],
       [   0,    0,    0, ..., 4589, 4481, 3186],
       ...,
       [   0,    0,    0, ..., 4589, 4481, 3186],
       [   0,    0,    0, ...,    0,    0, 1780],
       [   0,    0,    0, ...,  703, 2207, 3033]], dtype=int32)

In [32]:
Y_classes = np.argmax(Y_pred,axis=1)

In [33]:
Y_classes

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,Y_classes)

array([[3419,    0],
       [2616,    0]])

In [35]:
Y_pred=Y_pred.round()
Y_pred

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,Y_pred)

array([[3132,  287],
       [ 306, 2310]])

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9017398508699255